In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import json

In [2]:
#***          Start ib_insync (once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [3]:
#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

In [4]:
#******          Symbols Master      *******
#___________________________________________


# NSE symbols from https://www.nseindia.com/products/content/derivatives/equities/fo_underlying_home.htm

index_symbols = 'FTSE100,INDIAVIX,NIFTYCPSE,NIFTY,NIFTYIT,BANKNIFTY,NIFTYMID50,NIFTYPSE,NIFTYINFRA'.split(',')

symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

In [5]:
#......    !Data Limiter   .........
symbols=symbols[0:10]
#___________________________________

In [6]:
# Rectified for IBKR
isymbols = [i[0:9] for i in symbols]
isymbols = [s.replace('%26','') for s in isymbols]

# Rectified for 5paisa
psymbols = [s.replace('%26', '&') for s in symbols]

In [7]:
%%time
#***     Dictionaries and extracts ***
#_____________________________________

# URLs
urlbase = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
nseurl = urlbase + "get_quote/GetQuote.jsp?symbol="

# Scrape
nsestocks = [nseurl+i for i in symbols]
nserequests = [requests.get(b).text for b in nsestocks]
bs_nses = [BeautifulSoup(s, 'html.parser') for s in nserequests]
nse_data = [f.find(id='responseDiv').text.strip() for f in bs_nses]
nse_dict = [json.loads(bd) for bd in nse_data]

# NSE dictionary extracts
nse_margin = {i['data'][0]['symbol']: float(i['data'][0]['applicableMargin'])/1000 for i in nse_dict}
nse_hi52 = {i['data'][0]['symbol']: float((i['data'][0]['high52']).replace(',','')) for i in nse_dict}
nse_lo52 = {i['data'][0]['symbol']: float((i['data'][0]['low52']).replace(',','')) for i in nse_dict}
nse_lastprice = {i['data'][0]['symbol']: float((i['data'][0]['lastPrice']).replace(',','')) for i in nse_dict}

# MLot and TotalMargin from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)
paisa_small = paisa[['Symbol', 'Mlot', 'TotMgnPerLt']].sort_values('Symbol')

paisa_dict = paisa_small.set_index('Symbol').T.to_dict()

Wall time: 8.83 s


In [8]:
%%time
#***   Determine Standard Deviation    ***
#_________________________________________

market = 'NSE'

# qualify contracts
contracts = [Stock(symbol=s, exchange=market) for s in isymbols]
qual_stocks = ib.qualifyContracts(*contracts)

# make a historical dictionary
hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D', 
                                    barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)) for c in contracts])

data = [v for k, v in hist_dict.items()]

sd252 = [catch(lambda: np.std([i.close for i in data[j]][-252:], ddof=1)) 
         for j, k in enumerate(data)] # 252 trading days in a year

sd252_dict = dict(zip(isymbols, sd252))

Wall time: 7.2 s


In [9]:
%%time
#***    Get the RoM  ****
#________________________

# Option chains for qualified stocks
id_symbols = [(s.conId, s.symbol) for s in qual_stocks]

chains = [(symbol, 
           ib.reqSecDefOptParams(futFopExchange='', underlyingConId=cid, underlyingSecType='STK', underlyingSymbol=symbol)) 
          for cid, symbol in id_symbols]

str_exp_dict = dict([(x[0], (sorted(x[1][0].expirations), sorted(x[1][0].strikes))) for x in chains])


# Conversion dictionary for symbols
i2n_symbols = dict(zip(isymbols, psymbols)) #psymbols used because '%26' is converted to '&'...for nse_lastprice

# Filter the strikes
puts_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i < (nse_lastprice[i2n_symbols[k]]- sd252_dict[k]*2)]

calls_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i > (nse_lastprice[i2n_symbols[k]] + sd252_dict[k]*2)]

puts_clist = [{k: ('NSE', e, v[1], 'P')} for puts_dict in puts_list for k, v in puts_dict.items() for e in v[0]]
calls_clist = [{k: ('NSE', e, v[1], 'C')} for calls_dict in calls_list for k, v in calls_dict.items() for e in v[0]]

for puts_dict in puts_clist:
    for k, v in puts_dict.items():
        print(Option(symbol=k, exchange=v[0], lastTradeDateOrContractMonth=v[1], strike=v[2], right=v[3]))

put_option_contracts = [Option(symbol=k, exchange=v[0], lastTradeDateOrContractMonth=v[1], strike=v[2], right=v[3]) 
for puts_dict in puts_clist for k, v in puts_dict.items()]


# Qualify the put contracts
qual_puts = [ib.qualifyContracts(c) for c in put_option_contracts]

qual_puts = [l for i in qual_puts for l in i] # Remove the empty [] entries

live_put_ticks = [t for i in range(0, len(qual_puts), 100) for t in ib.reqTickers(*qual_puts[i:i+100])]

Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.

ERROR:ib_insync.wrapper:Error 200, reqId 34: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 35: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 37: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=880.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContract

ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1080.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 71: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1100.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 200: No security definition has been found for the request, contract: Option(symbol='ADANIPOWE', lastTradeDateOrContractMonth='20181129', strike=2.5, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ADANIPOWE', lastTradeDateOrContractMonth='20181129', strike=2.5, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 217: No security definition has been found for the request, contract: Option(symbol='AJANTPHAR', last

ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 311: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 313: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=105.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=105.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 314: No security definition has been found for the request, contract: Option(symb

Wall time: 3min 52s


In [10]:
live_put_ticks

[Ticker(contract=Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC'), time=datetime.datetime(2018, 9, 17, 12, 53, 13, 137097, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC'), time=datetime.datetime(2018, 9, 17, 12, 53, 13, 137097, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', c

In [11]:
bars = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='1 D', 
                                    barSizeSetting='1 day',  whatToShow='BID_ASK', useRTH=True) for c in qual_puts]

ERROR:ib_insync.wrapper:Error 162, reqId 743: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP860PE@NSE Ask, contract: Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 744: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP880PE@NSE Ask, contract: Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 745: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP900PE@NSE Ask, contract: Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', cur

ERROR:ib_insync.wrapper:Error 162, reqId 778: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18SEP5PE@NSE Ask, contract: Option(conId=332379980, symbol='ADANIENT', lastTradeDateOrContractMonth='20180927', strike=5.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18SEP5PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 779: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18OCT5PE@NSE Bid, contract: Option(conId=332380253, symbol='ADANIENT', lastTradeDateOrContractMonth='20181025', strike=5.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18OCT5PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 780: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18NOV5PE@NSE Ask, contract: Option(conId=332380662, symbol='ADANIENT', lastTradeDateOrContractMonth='20181129', strike=5.0, right='P', m

ERROR:ib_insync.wrapper:Error 162, reqId 801: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18NOV40PE@NSE Bid, contract: Option(conId=332380717, symbol='ADANIENT', lastTradeDateOrContractMonth='20181129', strike=40.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18NOV40PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 802: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18SEP45PE@NSE Ask, contract: Option(conId=332378791, symbol='ADANIENT', lastTradeDateOrContractMonth='20180927', strike=45.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18SEP45PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 803: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18OCT45PE@NSE Ask, contract: Option(conId=332380318, symbol='ADANIENT', lastTradeDateOrContractMonth='20181025', strike=45.0, righ

ERROR:ib_insync.wrapper:Error 162, reqId 824: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18OCT80PE@NSE Bid, contract: Option(conId=332380373, symbol='ADANIENT', lastTradeDateOrContractMonth='20181025', strike=80.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18OCT80PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 825: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18NOV80PE@NSE Ask, contract: Option(conId=332380780, symbol='ADANIENT', lastTradeDateOrContractMonth='20181129', strike=80.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIENT18NOV80PE', tradingClass='ADANIENT')
ERROR:ib_insync.wrapper:Error 162, reqId 826: Historical Market Data Service error message:HMDS query returned no data: ADANIENT18SEP85PE@NSE Bid, contract: Option(conId=332378991, symbol='ADANIENT', lastTradeDateOrContractMonth='20180927', strike=85.0, righ

ERROR:ib_insync.wrapper:Error 162, reqId 846: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18NOV220PE@NSE Ask, contract: Option(conId=331907488, symbol='ADANIPORT', lastTradeDateOrContractMonth='20181129', strike=220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18NOV220PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 847: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18SEP230PE@NSE Bid, contract: Option(conId=324365349, symbol='ADANIPORT', lastTradeDateOrContractMonth='20180927', strike=230.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18SEP230PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 848: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18OCT230PE@NSE Ask, contract: Option(conId=327839685, symbol='ADANIPORT', lastTradeDateOrContractMonth='2018

...

ERROR:ib_insync.wrapper:Error 162, reqId 941: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18OCT480PE@NSE Ask, contract: Option(conId=327683637, symbol='AMARAJABA', lastTradeDateOrContractMonth='20181025', strike=480.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18OCT480PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 942: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18NOV480PE@NSE Bid, contract: Option(conId=332626646, symbol='AMARAJABA', lastTradeDateOrContractMonth='20181129', strike=480.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18NOV480PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 943: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18SEP500PE@NSE Ask, contract: Option(conId=324353237, symbol='AMARAJABA', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 966: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18NOV640PE@NSE Bid, contract: Option(conId=331905273, symbol='AMARAJABA', lastTradeDateOrContractMonth='20181129', strike=640.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18NOV640PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 969: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18NOV660PE@NSE Ask, contract: Option(conId=331905282, symbol='AMARAJABA', lastTradeDateOrContractMonth='20181129', strike=660.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18NOV660PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 972: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18NOV680PE@NSE Ask, contract: Option(conId=331905288, symbol='AMARAJABA', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 995: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18OCT155PE@NSE Ask, contract: Option(conId=327661974, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=155.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMBUJACEM18OCT155PE', tradingClass='AMBUJACEM')
ERROR:ib_insync.wrapper:Error 162, reqId 996: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18NOV155PE@NSE Ask, contract: Option(conId=331929410, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', strike=155.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMBUJACEM18NOV155PE', tradingClass='AMBUJACEM')
ERROR:ib_insync.wrapper:Error 162, reqId 998: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18OCT160PE@NSE Ask, contract: Option(conId=327661980, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', 

ERROR:ib_insync.wrapper:Error 162, reqId 1020: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT640PE@NSE Ask, contract: Option(conId=327654573, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181025', strike=640.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18OCT640PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 1021: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18SEP660PE@NSE Ask, contract: Option(conId=324389455, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20180927', strike=660.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18SEP660PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 1022: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT660PE@NSE Bid, contract: Option(conId=327654582, symbol='APOLLOHOS', lastTradeDateOrContractMonth='2

ERROR:ib_insync.wrapper:Error 162, reqId 1044: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT820PE@NSE Ask, contract: Option(conId=327654647, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181025', strike=820.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18OCT820PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 1045: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18NOV820PE@NSE Ask, contract: Option(conId=331915952, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181129', strike=820.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18NOV820PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 1048: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18NOV840PE@NSE Ask, contract: Option(conId=331915962, symbol='APOLLOHOS', lastTradeDateOrContractMonth='2

In [16]:
bars

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 17), open=3.0, high=3.0, low=3.0, close=3.0, volume=-1, barCount=-1, average=-1.0)],
 [BarData(date=datetime.date(2018, 9, 17), open=1.0, high=1.0, low=1.0, close=1.0, volume=-1, barCount=-1, average=-1.0)],
 [],
 [BarData(date=datetime.date(2018, 9, 17), open=1.0, high=1.0, low=1.0, close=1.0, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 17), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 17), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [BarData(date=datetime.date(2018, 9, 17), open=0.05, high=0.05, low=0.05, close=0.05, volume=-1, barCount=-1, average=-1.0)],
 [BarData(date=datetime.date(2018, 9, 17), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [BarData(dat

In [31]:
for s, b in zip(qual_puts, bars):
    print(s.symbol, s.lastTradeDateOrContractMonth, s.strike, [(a.open, a.high, a.low, a.close) for a in b])

ACC 20180927 860.0 []
ACC 20180927 880.0 []
ACC 20180927 900.0 []
ACC 20180927 920.0 []
ACC 20180927 940.0 []
ACC 20180927 960.0 []
ACC 20180927 980.0 []
ACC 20180927 1000.0 []
ACC 20180927 1020.0 []
ACC 20180927 1040.0 []
ACC 20180927 1060.0 []
ACC 20180927 1080.0 [(3.0, 3.0, 3.0, 3.0)]
ACC 20180927 1100.0 [(1.0, 1.0, 1.0, 1.0)]
ACC 20181025 1100.0 []
ACC 20180927 1120.0 [(1.0, 1.0, 1.0, 1.0)]
ACC 20181025 1120.0 []
ACC 20181129 1120.0 []
ACC 20180927 1140.0 []
ACC 20181025 1140.0 []
ACC 20181129 1140.0 []
ACC 20180927 1160.0 []
ACC 20181025 1160.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1160.0 []
ACC 20180927 1180.0 []
ACC 20181025 1180.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1180.0 []
ACC 20180927 1200.0 [(0.05, 0.05, 0.05, 0.05)]
ACC 20181025 1200.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1200.0 []
ACC 20180927 1220.0 [(0.05, 0.05, 0.05, 0.05)]
ACC 20181025 1220.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1220.0 []
ACC 20180927 1240.0 [(0.05, 0.05, 0.05, 0.05)]
ACC 20181025 1240.0 

In [22]:
qual_puts

[Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC'),
 Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC'),
 Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP900PE', tradingClass='ACC'),
 Option(conId=324346574, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP920PE', tradingClass='ACC'),
 Option(conId=324346581, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP940PE', tradin

In [12]:
for i in qual_puts:
    print(i, paisa_dict[i.symbol]['Mlot'], 'Sell')

Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC') 400.0 Sell
Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP900PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346574, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP920PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346581, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', 

KeyError: 'ADANIPORT'

# Things to do...
* get option prices
* make qual_calls
* merge with margin to find RoM
* sort on RoM
* place orders
* pickle!

In [ ]:
ib.disconnect()